In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.utils import class_weight

# Paths for your datasets
train_dir = r"/content/dataset_testing_final/train"
val_dir = r"/content/dataset_testing_final/valid"

# Count images per class for the training data
class_counts = {}
for class_name in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        count = len([fname for fname in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, fname))])
        class_counts[class_name] = count

# Plot the class counts as a bar graph
plt.figure(figsize=(10, 6))
plt.bar(class_counts.keys(), class_counts.values(), color='skyblue')
plt.xlabel('Classes')
plt.ylabel('Number of Images')
plt.title('Number of Images per Class in Training Data')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Define ImageDataGenerator for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,  # Increased rotation
    width_shift_range=0.3,  # Increased shift
    height_shift_range=0.3,
    shear_range=0.3,  # Increased shear
    zoom_range=0.3,  # Increased zoom
    horizontal_flip=True,
    vertical_flip=True  # Added vertical flip
)

# Define ImageDataGenerator for validation (only rescaling)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Calculate class weights
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = dict(enumerate(class_weights))

# Build the CNN model
model = Sequential([
    # First block
    Conv2D(32, (3,3), padding='same', input_shape=(128, 128, 3)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Conv2D(32, (3,3), padding='same'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Second block
    Conv2D(64, (3,3), padding='same'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Conv2D(64, (3,3), padding='same'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Third block
    Conv2D(128, (3,3), padding='same'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Conv2D(128, (3,3), padding='same'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Fourth block (new)
    Conv2D(256, (3,3), padding='same'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Conv2D(256, (3,3), padding='same'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Global pooling layer
    GlobalAveragePooling2D(),

    # Fully connected layers
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),  # Increased patience
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)  # Increased patience
]

# Train the model
history = model.fit(
    train_generator,
    epochs=50,  # Increased epochs
    validation_data=val_generator,
    callbacks=callbacks,
    class_weight=class_weights_dict  # Use class weights
)

# Plot training and validation accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.figure(figsize=(14, 6))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo-', label='Training Accuracy')
plt.plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()
